## Documentation

To read more about analyzers, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-analyze.html).

![analyzer_api](../images/analyzer_api.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'WpOI-sfBSXe9aaVkWGIQnQ',
 'name': '61929d733ddf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


## 1. Character filters

Read more about them [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-charfilters.html).

### 1.1. HTML Strip Character Filter

In [4]:
resoponse = es.indices.analyze(
     body={
         "char_filter": [ "html_strip" ],
         "text": "<p>This is a test</p>"
     }
)
# OR

response = es.indices.analyze(
    char_filter=["html_strip"],
    text="<p>This is a test</p>"
)
pprint(resoponse.body)

{'tokens': [{'end_offset': 21,
             'position': 0,
             'start_offset': 0,
             'token': '\nThis is a test\n',
             'type': 'word'}]}


### 1.2. Mapping character filter

In [5]:
response = es.indices.analyze(
    tokenizer="keyword",
    char_filter=[
        {
            "type": "mapping",
            "mappings": [
                "٠ => 0",
                "١ => 1",
                "٢ => 2",
                "٣ => 3",
                "٤ => 4",
                "٥ => 5",
                "٦ => 6",
                "٧ => 7",
                "٨ => 8",
                "٩ => 9"
            ]
        }
    ],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
pprint(response.body)

{'tokens': [{'end_offset': 37,
             'position': 0,
             'start_offset': 0,
             'token': 'I saw comet Tsuchinshan Atlas in 2024',
             'type': 'word'}]}


## 2. Tokenizer

Read more about tokenizers [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenizers.html).

### 2.1. Standard

In [ ]:
response = es.indices.analyze(
    tokenizer="standard",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone.",
)
pprint(response.body)

{'tokens': [{'end_offset': 3,
             'position': 0,
             'start_offset': 0,
             'token': 'The',
             'type': '<ALPHANUM>'},
            {'end_offset': 5,
             'position': 1,
             'start_offset': 4,
             'token': '2',
             'type': '<NUM>'},
            {'end_offset': 11,
             'position': 2,
             'start_offset': 6,
             'token': 'QUICK',
             'type': '<ALPHANUM>'},
            {'end_offset': 17,
             'position': 3,
             'start_offset': 12,
             'token': 'Brown',
             'type': '<ALPHANUM>'},
            {'end_offset': 23,
             'position': 4,
             'start_offset': 18,
             'token': 'Foxes',
             'type': '<ALPHANUM>'},
            {'end_offset': 30,
             'position': 5,
             'start_offset': 24,
             'token': 'jumped',
             'type': '<ALPHANUM>'},
            {'end_offset': 35,
             'position': 6,
  

### 2.2. Lowercase

In [9]:
response = es.indices.analyze(
    tokenizer="lowercase",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone.",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}', Type: {token['type']}")

Token: 'the', Type: word
Token: 'quick', Type: word
Token: 'brown', Type: word
Token: 'foxes', Type: word
Token: 'jumped', Type: word
Token: 'over', Type: word
Token: 'the', Type: word
Token: 'lazy', Type: word
Token: 'dog', Type: word
Token: 's', Type: word
Token: 'bone', Type: word


### 2.3. Whitespace

In [25]:
response = es.indices.analyze(
    tokenizer="whitespace",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone.",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}', Type: {token['type']}")

Token: 'The', Type: word
Token: '2', Type: word
Token: 'QUICK', Type: word
Token: 'Brown-Foxes', Type: word
Token: 'jumped', Type: word
Token: 'over', Type: word
Token: 'the', Type: word
Token: 'lazy', Type: word
Token: 'dog's', Type: word
Token: 'bone.', Type: word


## 3. Token filter

Read more about token filters [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenfilters.html).

### 3.1. Apostrophe

In [10]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=[
        "apostrophe"
    ],
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone.",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'The'
Token: '2'
Token: 'QUICK'
Token: 'Brown'
Token: 'Foxes'
Token: 'jumped'
Token: 'over'
Token: 'the'
Token: 'lazy'
Token: 'dog'
Token: 'bone'


### 3.2. Decimal digit

In [11]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=[
        "decimal_digit"
    ],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'I'
Token: 'saw'
Token: 'comet'
Token: 'Tsuchinshan'
Token: 'Atlas'
Token: 'in'
Token: '2024'


### 3.3. Reverse

In [30]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=[
        "reverse"
    ],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'I'
Token: 'was'
Token: 'temoc'
Token: 'nahsnihcusT'
Token: 'saltA'
Token: 'ni'
Token: '٤٢٠٢'


## 4. Built-in analyzers

Read more about token filters [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-analyzers.html).

### 4.1. Standard

In [34]:
response = es.indices.analyze(
    analyzer="standard",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'i'
Token: 'saw'
Token: 'comet'
Token: 'tsuchinshan'
Token: 'atlas'
Token: 'in'
Token: '٢٠٢٤'


### 4.2. Stop

In [35]:
response = es.indices.analyze(
    analyzer="stop",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'i'
Token: 'saw'
Token: 'comet'
Token: 'tsuchinshan'
Token: 'atlas'


### 4.3. Keyword

In [36]:
response = es.indices.analyze(
    analyzer="keyword",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤",
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'I saw comet Tsuchinshan Atlas in ٢٠٢٤'


## 5. Index time VS Search time analysis

### 5.1. Index time

Index-time analysis transforms text before it's stored in the index. In this example, let's create an index with an analyzer that lowercases text, removes HTML tags, and replaces ampersands (&) with the word "and."

In [40]:
index_name = "index_time_example"
settings = {
    "settings": {
        "analysis": {
            "char_filter": {
                "ampersand_replacement": {
                    "type": "mapping",
                    "mappings": ["& => and"]
                }
            },
            "analyzer": {
                "custom_index_analyzer": {
                    "type": "custom",
                    "char_filter": ["html_strip", "ampersand_replacement"],
                    "tokenizer": "standard",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "custom_index_analyzer"
            }
        }
    }
}

es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=settings)

document = {
    "content": "Visit my website https://myuniversehub.com/ & like some images!"}
response = es.index(index=index_name, id=1, body=document)
pprint(response.body)

{'_id': '1',
 '_index': 'index_time_example',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


When searching for the document, you'll notice that the content appears unchanged. This is expected because Elasticsearch stores the transformed tokens in an inverted index for searching purposes, while keeping the original document intact in the `_source` field.

In [43]:
response = es.search(index=index_name, body={"query": {"match_all": {}}})
hits = response.body["hits"]["hits"]

for hit in hits:
    print(hit["_source"])

{'content': 'Visit my website https://myuniversehub.com/ & like some images!'}


We can verify that the custom analyzer is working by applying it to the document like this.

In [45]:
response = es.indices.analyze(
    index=index_name,
    body={
        "field": "content",
        "text": "Visit my website https://myuniversehub.com/ & like some images!"
    }
)

tokens = response.body["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}'")

Token: 'visit'
Token: 'my'
Token: 'website'
Token: 'https'
Token: 'myuniversehub.com'
Token: 'and'
Token: 'like'
Token: 'some'
Token: 'images'


### 5.2. Search time

Search-time analysis transforms text only when a search query is performed, not when data is indexed. In this example, we’ll perform a search with a search-time analyzer that transforms text differently (e.g., it lowercases and removes stop words).

In [ ]:
response = es.search(index=index_name, body={
    "query": {
        "match": { # match is used for full-text search
            "content": {
                "query": "myuniversehub.com",
                "analyzer": "standard"  # Using a different analyzer than the one used at index time
            }
        }
    }
})

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])

{'content': 'Visit my website https://myuniversehub.com/ & like some images!'}


You can also use a `term` query to match exact terms. Since `myuniversehub.com` exists exactly as-is in the document, this query will return the document in the results.

In [56]:
response = es.search(index=index_name, body={
    "query": {
        "term": {  # term is used for exact matches
            "content": {
                "value": "myuniversehub.com",
            }
        }
    }
})

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])

{'content': 'Visit my website https://myuniversehub.com/ & like some images!'}


In this case, `MYUNIVERSEHUB.com` does not appear in the document, so no results are returned.

In [57]:
response = es.search(index=index_name, body={
    "query": {
        "term": {  # term is used for exact matches
            "content": {
                "value": "MYUNIVERSEHUB.com",
            }
        }
    }
})

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])